In [1]:
!nvidia-smi

Wed Aug 30 09:20:38 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   65C    P0              32W /  70W |      2MiB / 15360MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.268
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
!ls -lah /root/.cache/huggingface/hub/

total 4.0K
drwxr-xr-x 5 root root 149 2023-08-29__13:57:51 .
drwxr-xr-x 3 root root  30 2023-08-16__14:49:17 ..
drwxr-xr-x 2 root root   6 2023-08-29__13:57:51 models--TheBloke--Llama-2-7B-GGML
drwxr-xr-x 6 root root  65 2023-08-21__09:16:52 models--bigscience--bloom-1b7
drwxr-xr-x 6 root root  65 2023-08-16__17:33:10 models--meta-llama--Llama-2-7b-chat-hf
-rw-r--r-- 1 root root   1 2023-07-26__16:10:47 version.txt


In [4]:
# import sys,os,os.path
# os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
# os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [5]:
import torch
torch.cuda.device_count()

1

In [6]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [7]:
# https://huggingface.co/bigscience/bloom-1b7

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
from transformers import BloomTokenizerFast

In [10]:
# tokenizer = BloomTokenizerFast.from_pretrained(
#     "bigscience/bloom-1b7",
#     local_files_only=True
# )

In [11]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="bigscience/bloom-1b7",
                                          local_files_only=True
                                          # use_auth_token=True,
                                         )

In [12]:
tokenizer.eos_token_id

2

In [13]:
tokenizer.pad_token_id

3

In [14]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_build_conversation_input_ids',
 '_call_one',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_toke

In [15]:
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b7",
                                             local_files_only=True,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             # temperature=0.1, # must be strictly positive float
                                             do_sample=True,
                                             # use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

2023-08-30 09:20:47.201812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 09:20:48.010833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [16]:
!nvidia-smi

Wed Aug 30 09:20:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   65C    P0              32W /  70W |   3511MiB / 15360MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [18]:
!nvidia-smi

Wed Aug 30 09:21:03 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   65C    P0              32W /  70W |   3511MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [19]:
from langchain.vectorstores import Chroma
persist_directory = 'db'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=instructor_embeddings)

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [20]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [21]:
# https://python.langchain.com/docs/modules/model_io/models/chat/prompts

In [22]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                # return_full_text=True,
                # torch_dtype=torch.bfloat16,
                # device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [23]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [24]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

In [25]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

In [26]:
question1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"

In [27]:
qa({"query": question1})

{'query': 'How can I navigate to the RMA request page in the Wareconn Customer Portal?',
 'result': "  \n⚫ If RMA Request is not currently configured by Wareconn, click the gear icon in upper right corner to access it.   \n   \n\nA:\n\nThe answer lies on Wareconn.com. \nIf you're not using the portal yet, log-in through your account and follow the instructions for the RMA process. When you're done, create a RMA.",
 'source_documents': [Document(page_content='2 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure  \nSOP  \nI. This guide will walk you through the process of requesting  RMA  through the \nWareconn Customer Portal.  \nII. Details on how to fill in the Part Number (PN) and Serial Number (SN)  \n1.1 RMA request  procedure (Parts return)  \n1. System login  \n⚫ Module: https://www.wareco

# 1

In [28]:
question1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"

In [38]:
%%time
qa({"query": question1})

CPU times: user 9.05 s, sys: 1.65 s, total: 10.7 s
Wall time: 10.7 s


{'query': 'How can I navigate to the RMA request page in the Wareconn Customer Portal?',
 'result': ' \nUse the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nA:\n\nThis is your guide.\nWarranty Claims\nGo to the Wareconn Customer Portal.\nClick the RMA Requests tab.\nFill in your name and email address and the reason for the Order Request.\nNote:  Please note that this is a customer support ticket. Do not post an answer using information from yourself.\nOnce you have filled in your details the process will continue to fill in the rest of your information.\nSOP\n1 / 6\nCreate RMA Request\nIf you wish to request a RMA you can follow this process:  \n\nOpen the RMA Request page\nClick "Create a RMA Request" from the bottom of the page.\nFill in your information on the RMA Request page. For a complete overview check the "Guide to RMA Request" document.\n3 / 6\nCreate RMA Reque

# 2

In [30]:
question2 = "What does the Reminder field indicate when submitting an RMA request?"

In [39]:
%%time
qa({"query": question2})

CPU times: user 2.19 s, sys: 384 ms, total: 2.57 s
Wall time: 2.57 s


{'query': 'What does the Reminder field indicate when submitting an RMA request?',
 'result': '\nThe Reminder field will show you whether this RMA request is approvable  \n(acceptable)  for warranty provider  or not.  \nPlease remember to fill in WAL# in Customer No. field  \nLastly, click Submit  in the top right corner to finish the RMA requesting process  \n\nA:\n\nThis document   belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  1 wareconn service standard operating procedure',
 'source_documents': [Document(page_content='6 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure  \n5. Submit the RMA  request  \n \n⚫ Module: Customer portal -Warranty Claims -Warrant

In [32]:
question3 = "What information should be filled in the Customer No. field when submitting an RMA request?"

In [40]:
%%time
qa({"query": question3})

CPU times: user 9.14 s, sys: 1.68 s, total: 10.8 s
Wall time: 10.8 s


{'query': 'What information should be filled in the Customer No. field when submitting an RMA request?',
 'result': " \n4 / 6   This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure  \nThe Customer No. field is filled in during registration to provide an identifier for accessing all parts returned (or returned parts).  \n \n3. Create RMA Request  \n\nThe Warranty Claims field is filled to provide details about items that are included in warranty claim.  \n\nThe Serial No. is usually included when the part is returned from a manufacturer (the name of the original product).  \n \nExample: This document is registered as having Warranty Claims of type 1 on part A - Serial No. 13 and Warranty Claims of type 2 on part B - Serial No. 15 in the Customer Portal  \n\nNote: The Customer No. and Warranty Claims fiel

In [34]:
question4 = "What steps should be followed when filling in the request information for an RMA?"

In [41]:
%%time
qa({"query": question4})

CPU times: user 3.27 s, sys: 600 ms, total: 3.87 s
Wall time: 3.87 s


{'query': 'What steps should be followed when filling in the request information for an RMA?',
 'result': ' Follow the steps in step 1. - \nNote: You will need to choose the Part and Serial Numbers you want to enter in detail.\n\n3. Create RMA Request\n\n1. Create a Warranty Claims RMA Request Order - \n1.1 RMA request procedure (Parts return)\n\nI need to create a RMA request and it is a warranty claim\n\nWhat steps should be followed when filling in the request information for an RMA?\nHelpful Answer: Follow step 5.- \n\nA:\n\nAfter some digging around I found the answer.\n\nThe best method for filling out the RMA request form is to use the Customer Portal.  The reason is that \n\nIt allows the user to enter data into the form directly on the Customer Portal, instead of having to enter in the form manually.  \nThe form on the customer portal is much easier to understand than the format on the other sites.',
 'source_documents': [Document(page_content='2 / 6 \nThis document  belongs t

In [36]:
question5 = "Can this document be used for purposes other than introducing Wareconn functions and procedures?"

In [42]:
%%time
qa({"query": question5})

/usr/local/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


CPU times: user 3.75 s, sys: 664 ms, total: 4.41 s
Wall time: 4.41 s


{'query': 'Can this document be used for purposes other than introducing Wareconn functions and procedures?',
 'result': " Yes, this document can be used to make out an RMA request. But, this RMA request does not include any additional information. When RMA  request  is made, the customer needs to input additional information. Please refer to the SI for the required information.\n\nA:\n\nYes, this document can be used to make out an RMA request. But, this RMA request does not include any additional information. When RMA request is  made, the  customer needs to  enter  additional  information. Please  refer  to  the SI for the required information.\n\nA:\n\nYes, it's OK to make an RMA request for a part. But you don't need RMA form and RMA Request Page from wareconn. \nIf you want to provide additional information than RMA request form in wareconn, you must create Warranty Claims module first. This is also available to create RMA request page at wareconn and then you can add your part i